# Heat Map & Marker Cluster Map Tutorial

<p>In this tutorial, we will be looking at how to create a heat map and marker cluster map using Folium. </p>
<b>Heat Map</b>  


<img src="data/heatmap.png" height = "800" width = "800" align="left" />

<b>Marker Cluster Map</b>  
<img src="data/markerclustermap.png" height = "800" width = "800" align="left" />

## Software & Packages

1. Python 3
2. Pandas - for data transformation
3. Folium - for map visualization

Install Pandas and Folium using  
(a) pip install pandas  
(b) pip install folium  
if you have not yet install them in your python environment  

### Import Packages

Let's import the required packages

In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

### The Data

The data is crawled from LTA DataMall Traffic Incident API at interval of 1 minute across a one week period in Feburary 2020. It consist of a timestamp to indicate the crawl time, type of traffic incident, latitude, longitude and a message to describe the incident.  

For the purpose of this tutorial, we are only looking at "Vehicle breakdown" and "Accident" incident type which makes up 7719 data points in total.

In [2]:
df = pd.read_csv('data/traffic-incident-tutorial.csv')
df.sort_values(by = ['type', 'unix_ts']).head()

,unix_ts,type,longitude,latitude,message
3,1581898147,Accident,103.859376,1.311657,(17/2)7:59 Accident on Jalan Besar (towards Ci...
7,1581898267,Accident,103.859376,1.311657,(17/2)7:59 Accident on Jalan Besar (towards Ci...
11,1581898387,Accident,103.859376,1.311657,(17/2)7:59 Accident on Jalan Besar (towards Ci...
14,1581898867,Accident,103.859376,1.311657,(17/2)7:59 Accident on Jalan Besar (towards Ci...
16,1581898987,Accident,103.859376,1.311657,(17/2)7:59 Accident on Jalan Besar (towards Ci...


### Data Cleaning

LTA publishes the incidents through a live feed and updates it every minute. Incident information will only be removed after an incident is resolved. As we crawled the data at interval of 1 minute, the same incident will appear at different crawled timestamps,Therefore we need to drop duplicated entries of the same incident.

Note that an the "message" column for an incident may change over time depending on how the incident evolves.

The first word token in the "message" column is the reported date and time of the incident, we can use this information to help us remove duplicates.  
We extract this information into a new column called "message_dt" and drop the entire row if it is duplicated across "type", "message_dt", "latitude" and "longitude".

We are left with 677 data points after removing duplicates.

In [3]:
df['message_dt'] = df['message'].apply(lambda x: str(x).split(' ')[0])
df_latlon =\
df.drop_duplicates(subset = ['type','message_dt', 'latitude', 'longitude']).loc[:,['latitude', 'longitude']]

Let's create a list of tuple, where each tuple contains a latitude and longitude of an incident.  
This will be used as input for mapping later.

In [4]:
latlon_list = list(zip(df_latlon['latitude'], df_latlon['longitude']))

### Let's Start Viz-ing!

#### Heat Map Viz

Folium map works on a layering principle where each layer contains some visual representation.  
We start off by creating a base layer, which we call name as "basemap".  
This layer consist of the map tile and we are using "OpenStreeMap".
The "location" parameter tells Folium which coordinates to center the map on.  
The "zoom_start" parameter sets the level of map zoom when the map is rendered. Users can subsequently adjust the zoom setting using the +/- buttons at the top left of the map

We render the map by simply running the assigned variable name "basemap".  
Notice that the incident data points are not plotted onto the map yet.

In [5]:
basemap = folium.Map(location= [1.3521, 103.8198], tiles = "OpenStreetMap",zoom_start = 12)
basemap

Next, let's add a new layer that contains the incident data points to the map.  
We create a "HeatMap" layer by defining the data points and other visualization parameters.  
The "radius" parameter tells Folium the maximum distance between 2 points to consider them as the same cluster on the heat map.  
The "gradient" parameters helps define the colour gradient at various intensity threshold on the heat map. 

We then add the "HeatMap" layer to the base layer.

In [6]:
HeatMap(data = latlon_list, 
        radius = 12,
        gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}
       ).add_to(basemap)

Now we can see the incident heatmap being overlay on the basemap.   

In [7]:
basemap

The map can also be saved as a .html file and opened in an internet browser. 

In [8]:
basemap.save('heatmap.html')

#### Marker Cluster Map Viz

We perform similar steps as above to create a marker cluster map.  
First we create a base layer which we name as "basemap".  
Then we create a MarkerCluster layer and add it to the basemap.  

Click on the clusters and watch them split!

In [9]:
basemap = folium.Map(location= [1.3521, 103.8198], tiles = "OpenStreetMap",zoom_start = 12)
MarkerCluster(locations = latlon_list).add_to(basemap)
basemap

The map can be saved as a .html file

In [10]:
basemap.save('markerclustermap.html')

I hope this tutorial is useful!  
Have fun viz-ing!